In [65]:

import re
import os
target_folder = '../docs/rust-tutorial'
META_TITLE = '---\ntitle: {}\n---\n'

In [73]:
count = 0
for file in os.listdir(target_folder):
    if not file.endswith('.md'):
        continue
    if not file.startswith('ch'):
        continue
    major_id = int(file[2:4]) + 1
    sub_id = int(file[5:7]) + 1
    file_path = os.path.join(target_folder, file)
    lines = open(file_path, 'r', encoding='utf-8').readlines()
    if lines[0].strip() == '---' and 'title' in lines[1]:
        title = lines[1].replace('title:', '').strip()
        title = 'title: {}.{} {}\n'.format(major_id, sub_id, title)
        lines[1] = title
        open(file_path, 'w', encoding='utf-8').writelines(lines)
count

0

In [2]:
text_path = '../docs/cpp-features/7.md'

def extract_cpp_code_blocks(markdown_text):  
    # 正则表达式匹配C++代码块  
    # 假设代码块以```cpp开始，并以```结束，可能包含多行  
    pattern = re.compile(r'```cpp\n([\s\S]*?)```', re.DOTALL)  # re.DOTALL使得.可以匹配任何字符，包括换行符  
      
    # 查找所有匹配项  
    matches = pattern.findall(markdown_text)  
      
    # 提取并返回代码块  
    return [match.strip() for match in matches]  # strip()去除每个代码块前后的空白字符

In [3]:
md_text = open(text_path, 'r', encoding='utf-8').read()
blocks = extract_cpp_code_blocks(md_text)

In [8]:


for file in os.listdir(target_folder):
    if not file.endswith('.md'):
        continue
    file_path = os.path.join(target_folder, file)
    lines = open(file_path, 'r', encoding='utf-8').readlines()
    first_line = lines[0].strip()
    if first_line.startswith('#'):
        title = first_line.lstrip('#').strip().replace('`', '').replace(':', '：')
        meta_title = META_TITLE.format(title)
        lines[0] = meta_title
        open(file_path, 'w', encoding='utf-8').writelines(lines)

In [23]:
# make config.ts
import json5

def group_by(array, id_gen):
    group_cluster = {}
    for element in array:
        id_name = id_gen(element)
        if id_name not in group_cluster:
            group_cluster[id_name] = []
        group_cluster[id_name].append(element)
    return group_cluster

md_files = os.listdir(target_folder)
md_cluster = group_by(md_files, lambda file: file.split('-')[0])

def get_meta_title(file):
    count = 0
    for line in open(file, 'r', encoding='utf-8'):
        if count == 0 and line.strip() != '---':
            return None
        if count == 1:
            title = line[6:].strip()
            return title
        count += 1

configs = []
for group_name in md_cluster:
    group_name: str
    if group_name.endswith('.md') or group_name in ['img', 'title']:
        continue
    mds = md_cluster[group_name]
    first_md = mds[0]
    group_title = get_meta_title(os.path.join(target_folder, first_md))
    print("\"/docs/rust-tutorial/" + first_md.split('.')[0] + "\",")
    configs.append({
        "text": group_title,
        "children": [md.split('.')[0] for md in mds]
    })

with open("config.json5", 'w', encoding='utf-8') as fp:
    fp.write(json5.dumps(configs, indent=4, ensure_ascii=False))

"/docs/rust-tutorial/appendix-00",
"/docs/rust-tutorial/ch00-00-introduction",
"/docs/rust-tutorial/ch01-00-getting-started",
"/docs/rust-tutorial/ch02-00-guessing-game-tutorial",
"/docs/rust-tutorial/ch03-00-common-programming-concepts",
"/docs/rust-tutorial/ch04-00-understanding-ownership",
"/docs/rust-tutorial/ch05-00-structs",
"/docs/rust-tutorial/ch06-00-enums",
"/docs/rust-tutorial/ch07-00-managing-growing-projects-with-packages-crates-and-modules",
"/docs/rust-tutorial/ch08-00-common-collections",
"/docs/rust-tutorial/ch09-00-error-handling",
"/docs/rust-tutorial/ch10-00-generics",
"/docs/rust-tutorial/ch11-00-testing",
"/docs/rust-tutorial/ch12-00-an-io-project",
"/docs/rust-tutorial/ch13-00-functional-features",
"/docs/rust-tutorial/ch14-00-more-about-cargo",
"/docs/rust-tutorial/ch15-00-smart-pointers",
"/docs/rust-tutorial/ch16-00-concurrency",
"/docs/rust-tutorial/ch17-00-oop",
"/docs/rust-tutorial/ch18-00-patterns",
"/docs/rust-tutorial/ch19-00-advanced-features",
"/docs/r

In [33]:
# 处理 mdBook 嵌入式语句
import re
import os

def replace_includes(markdown_content, base_dir):
    # 定义正则表达式模式来匹配 mdBook 的嵌入语法
    include_pattern = re.compile(r'{{#(include|rustdoc_include)\s+(.+?)\s*}}')

    # 定义替换函数
    def replace_match(match):
        directive = match.group(1)
        file_path = match.group(2)

        # 构建完整文件路径
        full_path = os.path.join(base_dir, file_path)

        # 检查文件是否存在
        if not os.path.exists(full_path):
            return f"**File not found: {file_path}**"

        # 读取文件内容
        with open(full_path, 'r', encoding='utf-8') as f:
            file_content = f.read()

        return file_content.strip()

    # 使用正则表达式替换所有匹配的内容
    return include_pattern.sub(replace_match, markdown_content)

def process_markdown_file(input_file, base_dir):
    # 读取输入文件内容
    with open(input_file, 'r', encoding='utf-8') as f:
        markdown_content = f.read()

    # 处理嵌入语法
    processed_content = replace_includes(markdown_content, base_dir)

    with open(input_file, 'w', encoding='utf-8') as f:
        f.write(processed_content)
      
for file in os.listdir(target_folder):
    file_path = os.path.join(target_folder, file)
    if os.path.isdir(file_path) or not file_path.endswith('.md'):
        continue

    process_markdown_file(file_path, target_folder)

../docs/rust-tutorial\appendix-00.md
../docs/rust-tutorial\appendix-01-keywords.md
../docs/rust-tutorial\appendix-02-operators.md
../docs/rust-tutorial\appendix-03-derivable-traits.md
../docs/rust-tutorial\appendix-04-useful-development-tools.md
../docs/rust-tutorial\appendix-05-editions.md
../docs/rust-tutorial\appendix-06-translation.md
../docs/rust-tutorial\appendix-07-nightly-rust.md
../docs/rust-tutorial\ch00-00-introduction.md
../docs/rust-tutorial\ch01-00-getting-started.md
../docs/rust-tutorial\ch01-01-installation.md
../docs/rust-tutorial\ch01-02-hello-world.md
../docs/rust-tutorial\ch01-03-hello-cargo.md
../docs/rust-tutorial\ch02-00-guessing-game-tutorial.md
../docs/rust-tutorial\ch03-00-common-programming-concepts.md
../docs/rust-tutorial\ch03-01-variables-and-mutability.md
../docs/rust-tutorial\ch03-02-data-types.md
../docs/rust-tutorial\ch03-03-how-functions-work.md
../docs/rust-tutorial\ch03-04-comments.md
../docs/rust-tutorial\ch03-05-control-flow.md
../docs/rust-tutori

In [64]:
# handle file not found

def get_file_content(base_dir, spls: list[str]):
    if spls.__len__() == 2:
        file, tag = spls[0], spls[1]
        file_path = os.path.join(base_dir, file)
        
        if tag.isdigit():
            lines = open(file_path, 'r', encoding='utf-8').readlines()
            return lines[int(tag) - 1]
        else:
            lines = open(file_path, 'r', encoding='utf-8').readlines()
            if tag == 'all':
                content = ''.join([l for l in lines if '//' not in l])
                return content
            
            start, end = None, None
            start_tag = 'ANCHOR: ' + tag
            end_tag = 'ANCHOR_END: ' + tag
            for i in range(lines.__len__()):
                line_content = lines[i].strip()
                if start_tag in line_content:
                    start = i
                if end_tag in line_content:
                    end = i
            
            if start is not None and end is not None:
                content = ''.join(lines[start + 1: end])
                return content
        
    elif spls.__len__() == 3:
        file, start, end = spls[0], spls[1], spls[2]
        file_path = os.path.join(base_dir, file)

        start = int(start) - 1
        lines = open(file_path, 'r', encoding='utf-8').readlines()
        
        if end == '':
            content = ''.join(lines[start: ])
        else:
            content = ''.join(lines[start: int(end)])
        
        return content
    return None

def file_not_found(input_file, base_dir):
    lines = open(input_file, 'r', encoding='utf-8').readlines()
    for i in range(lines.__len__()):
        line_content: str = lines[i]
        if '**File not found:' in line_content:
            link = line_content.split(' ')[-1].strip().strip('**')
            if ':' in link:
                spls = link.split(':')
                content = get_file_content(base_dir, spls)
                lines[i] = content
    
    open(input_file, 'w', encoding='utf-8').writelines(lines)

for file in os.listdir(target_folder):
    file_path = os.path.join(target_folder, file)
    if os.path.isdir(file_path) or not file_path.endswith('.md'):
        continue

    file_not_found(file_path, target_folder)